# Testing notebook

## Working the CSVs and Excel sheets

In [1]:
# Exploring the 'training' dataset
import pandas as pd
df = pd.read_csv("txt2sql_alerce_train_v3_1.csv")
df.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,rephrased_request,rephrased_request_gpt-3.5-turbo-0125_t0.4,rephrased_request_gpt-4o-2024-05-13_t0.2
0,13,Give me all the SNe that were first detected b...,"['object', 'probability']",\n-- mjd date for December = 59914.0\n-- mjd d...,\n-- Super Nova (SNe) is a large explosion tha...,"\nSELECT\n object.oid, probability.class_na...",simple,object,none,NaN,NaN,NaN
1,10,Get the object identifiers and probabilities i...,"['probability', 'object']",0,0,"\nSELECT\n sq1.oid, sq1.probability as SN_pro...",medium,object,simple,NaN,NaN,NaN
2,15,"Get the object identifiers, probabilities in t...","['object', 'probability', 'detection', 'magstat']",\n-- mjd date for September 01 = 60188.0\n-- m...,\n-- A fast riser is defined as an object whos...,"\nSELECT\n sq.oid, sq.probability, sq.candi...",advanced,other,multi,NaN,NaN,NaN
3,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,0,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,NaN,NaN,NaN
4,25,Query objects within 10 degress of the next po...,"['probability', 'object']",\n-- mjd date for February 01 = 59976.0\n-- mj...,0,"\nWITH catalog ( source_id, ra, dec) AS (\n ...",advanced,spatial,simple,NaN,NaN,NaN


In [2]:
df[df["request"] == "Find at most 10 cases where the feature called 'LinearTrend' is larger than 5 and their ZTF objects have a difference between last and first detection date larger than 1 day. Return the following columns. For the ZTF objects: ZTF identifier and number of detections; for their light curve classifier data at ranking 1: classifier version, class and probability; for the 'LinearTrend' feature: value and filter identifier"]["external_knowledge"].item()

"Since no feature version is specified, the combination between ZTF oid and fid can be non-unique in the 'feature' table output"

In [3]:
# Reading the excel file
excel = pd.read_excel("SQLusecases_alerce.xlsx", 
                      sheet_name="examples_alerce_usecasesV3_1")

# Cleaning the excel a bit
excel = excel.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])
excel.head()

,req_id,request,table_info,external_knowledge,domain_knowledge,gold_query,difficulty,type,nested_type,Set,python_format
0,0,Get objects that are likely to be YSOs (possib...,"['probability', 'feature']",\n-- feature.name can be 'Multiband_period'\n-...,\n-- Multiband_period: Period obtained using t...,"\nSELECT\n oid, probability, value, name, fid...",advanced,object,tree,Train,"sub_query_1='''\nSELECT\n feature.oid, prob_o..."
1,1,Get all the objects classified as AGN with a p...,"['object', 'probability', 'feature', 'magstat']",\n-- object.ndet represents the number of dete...,\n-- Amplitude: Half of the difference between...,"\nSELECT\n sq.oid, sq.value, sq.name, sq.fid ...",advanced,object,tree,Test,\nsub_query_object='''\nSELECT\n object.oid...
2,2,Give me the objects classified as YSO by their...,['probability'],NaN,NaN,"\nSELECT\n oid, probability\nFROM\n prob...",simple,object,none,Train,"query='''\nSELECT\n oid, probability\nFROM\..."
3,3,Give me the objects classified as YSO by the l...,"['object', 'probability']","\n-- last june in mjd date: [start=60096.0, en...",NaN,\nSELECT\n *\nFROM\n probability\nWHERE\...,simple,object,simple,Train,query=f'''\nSELECT\n *\nFROM\n probabili...
4,4,"Get the object identifier, candidate identifie...","['object', 'probability', 'magstat', 'detection']",\n-- mjd date for the start of the year 2019 =...,NaN,"\nSELECT\n sq.oid, sq.fid, sq.dmdt_first,\n ...",advanced,other,multi,Train,# objects classified as SN II with probability...


## Tests

### Gold values and query to test

In [4]:
# Select a query
query = "Find at most 50 ZTF objects that satisfy the following conditions: have a probability larger than 0.7 of being an AGN in the light curve classifier version 'hierarchical_rf_1.1.0', a distance to the nearest PS1 source lower than 2 arcsec, and a difference between the star/galaxy score from PS1 and the star/galaxy score from SExtractor lower than 0.5. Return the following columns. For the ZTF object: ZTF identifier and probability; for each epoch: candidate and filter identifier, reduced chi-square, star/galaxy score from SExtractor, and also the identifier, PS1 star/galaxy score and distance of the nearest PS1 source"

# Check if the query is in the excel file
if query in excel["request"].to_list():
    # Obtain the gold SQL query and Python query
    sql_gold = excel[excel["request"] == query]["gold_query"].item()
    python_gold = excel[excel["request"] == query]["python_format"].item()

    # Obtain the necessary tables
    gold_tables = excel[excel["request"] == query]["table_info"].item()

    # Print all in orderly fashion
    print("Gold values\n")
    print("Tables needed for the query:")
    print(gold_tables + "\n")
    print("SQL gold query:")
    print(sql_gold)
    print("Python gold query:\n")
    print(python_gold)
    
else:
    # Obtain the gold SQL query
    sql_gold = df[df["request"] == query]["gold_query"].item()

    # Obtain the necessary tables
    gold_tables = df[df["request"] == query]["table_info"].item()

    # Print all in orderly fashion
    print("Gold values\n")
    print("Tables needed for the query:")
    print(gold_tables + "\n")
    print("SQL gold query:")
    print(sql_gold)

Gold values

Tables needed for the query:
['object', 'probability', 'dataquality', 'ps1_ztf']

SQL gold query:
SELECT
    sq1.oid,
    sq1.probability, sq1.candid, sq1.fid,
    sq1.chipsf, sq1.classtar,
    sq2.objectidps1, sq2.sgscore1,
    sq2.distpsnr1
FROM
(
SELECT
    probability.oid,
    probability.probability, dataquality.candid,
    dataquality.fid, dataquality.chipsf,
    dataquality.classtar
FROM
    probability INNER JOIN
    dataquality
    ON probability.oid = dataquality.oid
WHERE
    classifier_name = 'lc_classifier'
    AND classifier_version = 'hierarchical_rf_1.1.0'
    AND class_name = 'AGN'
    AND probability > 0.7
) AS sq1
INNER JOIN
(
SELECT
    probability.oid, ps1_ztf.candid,
    ps1_ztf.objectidps1, ps1_ztf.sgscore1,
    ps1_ztf.distpsnr1
FROM
    probability INNER JOIN
    ps1_ztf
    ON probability.oid = ps1_ztf.oid
WHERE
    classifier_name = 'lc_classifier'
    AND classifier_version = 'hierarchical_rf_1.1.0'
    AND class_name = 'AGN'
    AND probability

In [5]:
# Running the gold query
from secret.config import SQL_URL
import requests
import sqlalchemy as sa

# Setup params for query engine
params = requests.get(SQL_URL).json()['params']
engine = sa.create_engine(f"postgresql+psycopg2://{params['user']}:{params['password']}@{params['host']}/{params['dbname']}")
engine.begin()

pd.read_sql_query(sql_gold, con=engine)

,oid,probability,candid,fid,chipsf,classtar,objectidps1,sgscore1,distpsnr1
0,ZTF19adcfkex,0.895584,1122215490915015013,1,0.844136,0.703,196461506589680576,0.000000,0.450575
1,ZTF19aayljdt,0.736064,889273400115010007,2,0.930399,0.710,93462548914078624,0.000000,0.212540
2,ZTF19aayljdt,0.736064,2663376350115010007,2,1.464525,0.722,93462551400022020,0.000000,0.408871
3,ZTF19aayljdt,0.736064,2708293920115010017,1,0.755105,0.728,93462551400022020,0.000000,0.255659
4,ZTF19aayljdt,0.736064,2708356890115010007,2,0.812704,0.870,93462551400022020,0.000000,0.025791
5,ZTF21acrahxc,0.705740,1796484212915015001,1,0.998092,0.787,121911687945259580,0.207875,0.651341
6,ZTF21aatxjsz,0.857960,1560264781615010001,2,0.916886,0.862,199211439285418180,0.000000,0.302456
7,ZTF19acmossq,0.756504,1058267391815010001,2,2.443726,0.937,180880922553841600,0.002500,0.432530
8,ZTF19acmossq,0.756504,2871339051815010002,2,1.635072,0.935,180880915583467520,0.002500,0.429585
9,ZTF19acmossq,0.756504,2871413911815010012,1,1.307080,0.809,180880915583467520,0.002500,0.188457


### Trying out the pipelines

In [ ]:
from pprint import pprint
from pipeline.main import run_pipeline
from tests import engine
engine.begin()

# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "gpt-4o"

# Format for the pipeline
format = "python"

# RAG parameters
max_tokens = 10000
size = 700
overlap = 300
quantity = 10

# Running the pipeline
result, total_usage, prompts = run_pipeline(query, model, max_tokens, size, 
                                            overlap, quantity, format, False, 
                                            engine, rag_pipe=False, 
                                            self_corr=True)
print("Resulting table:")
print(result)
print("Total usage of the pipeline:")
pprint(total_usage)

# The prompts used will be saved in this file
with open(f"prompts/examples/prompts_query_{model}.txt", "w") as f:
    f.write(str(prompts))

Running SQL exception: (psycopg2.errors.UndefinedColumn) column "sgscore1" does not exist
LINE 43:     ABS(sgscore1 - classtar) < 0.5
                 ^
HINT:  There is a column named "sgscore1" in table "d", but it cannot be referenced from this part of the query.

[SQL: 
SELECT
    o.oid AS ztf_identifier,
    prob.probability,
    det.candid AS candidate_identifier,
    det.fid AS filter_identifier,
    dq.chipsf AS reduced_chi_square,
    dq.classtar AS star_galaxy_score_sextractor,
    ps.objectidps1 AS ps1_identifier,
    ps.sgscore1 AS ps1_star_galaxy_score,
    ps.distpsnr1 AS distance_to_ps1
FROM
    (
SELECT
    p.oid
FROM
    (
SELECT DISTINCT
    oid
FROM
    probability
WHERE
    classifier_name = 'lc_classifier'
    AND classifier_version = 'hierarchical_rf_1.1.0'
    AND class_name = 'AGN'
    AND probability > 0.7
    AND ranking = 1
) AS p
    INNER JOIN (
SELECT
    oid, objectidps1, sgscore1, distpsnr1
FROM
    ps1_ztf
WHERE
    distpsnr1 < 2
) AS d ON p.oid = d.oid


Exception: Failed again: local variable 'e' referenced before assignment

In [ ]:
from pprint import pprint
from pipeline.main import run_pipeline, engine
engine.begin()

# Model to use
#model = "claude-3-5-sonnet-20240620"
model = "o1-preview"

# Format for the pipeline
format = "python"

# RAG parameters
max_tokens = 10000
size = 700
overlap = 300
quantity = 10

# Running the pipeline
result, total_usage, prompts = run_pipeline(query, model, max_tokens, size, 
                                            overlap, quantity, format, False, 
                                            engine, rag_pipe=True, 
                                            self_corr=True)
print("Resulting table:")
print(result)
print("Total usage of the pipeline:")
pprint(total_usage)

# The prompts used will be saved in this file
with open(f"prompts/examples/prompts_query_{model}.txt", "w") as f:
    f.write(str(prompts))

Tables needed: [object, detection, magstat]
Difficulty: advanced
```python
# Get oids of objects classified as 'SN II' with probability > 0.6
sub_query_probability = f'''
SELECT oid
FROM probability
WHERE classifier_name='lc_classifier'
AND class_name='SN II'
AND ranking=1
AND probability > 0.6
'''

# Get oids of objects with first observation between 2019-01-01 and 2022-12-31
# MJD range from 58484 (2019-01-01) to 59580 (2022-01-01)
sub_query_object = f'''
SELECT oid
FROM object
WHERE firstmjd BETWEEN 58484 AND 59946
'''

# Combine previous subqueries to get oids satisfying both class and date criteria
sub_query_oid1 = f'''
SELECT prob_oids.oid
FROM ({sub_query_probability}) AS prob_oids
INNER JOIN ({sub_query_object}) AS obj_oids
ON prob_oids.oid = obj_oids.oid
'''

# Get oids with more than 30 detections
sub_query_ndet = f'''
SELECT oid
FROM detection
GROUP BY oid
HAVING COUNT(*) > 30
'''

# Combine to get oids satisfying class, date, and number of detections criteria
sub_query_oid2

Exception: Failed again: local variable 'results' referenced before assignment